In [ ]:
import pandas as pd
from tqdm import tqdm # Só de bonito
import warnings

# Warning para ignorar mensagens de erro
warnings.simplefilter("ignore")

In [ ]:
# Importando os arquivos de PLD (o site da CCEE limita a extração a 6 meses)
primeiro_semestre = pd.read_csv("preco_horario_21_1s.csv", sep=";", decimal=",")
segundo_semestre = pd.read_csv("preco_horario_21_2s.csv", sep=";", decimal=",")

In [ ]:
# Criando as colunas para o merge
primeiro_semestre['merge'] = primeiro_semestre['Hora'].astype('str') + primeiro_semestre['Submercado']
segundo_semestre['merge'] = segundo_semestre['Hora'].astype('str') + segundo_semestre['Submercado']

In [ ]:
# Criando um df do ano
ano = pd.merge(primeiro_semestre, segundo_semestre, on="merge", suffixes=("", "_merge"))

In [ ]:
# Removendo as colunas que não fazem mais sentido
ano = ano.drop(['Hora_merge', 'Submercado_merge', 'merge'], axis=1)

In [ ]:
# Criando a indexação das datas
colunas = ano.columns
valores_a_remover = ['Hora', 'Submercado']
colunas_data = colunas.drop(valores_a_remover)

In [ ]:
# Trasnpondo as datas e criando um df
pld_ano = pd.DataFrame([])
for data in tqdm(colunas_data):
    df = ano[['Hora', 'Submercado', data]]
    df['Data'] = str(data)
    df.rename(columns={data: "PLD"}, inplace=True)
    df.reset_index(drop=True, inplace=True)
    pld_ano = pd.concat([pld_ano, df], ignore_index=True)

In [ ]:
# Convertendo a coluna Data em datetime
pld_ano['Data'] = pld_ano['Data'].astype('datetime64[ns]')

In [ ]:
# Exportando o arquivo
pld_ano.to_excel("pld_2021.xlsx", index=False)